<a href="https://colab.research.google.com/github/ckesmay/is4487/blob/main/Module6_Script_v1_5_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IS 4487 Module 5 Script

## Objectives

What do you need to know for this module's project?

1. Fit a tree model using all the available predictors. 
2. Create a confusion matrix  and identify numbers of TP, FP, TN and FN.
3. Estimate profit (benefits - costs) using a defined cost-benefit matrix and the above confusion matrix.
4. Estimate profit for a strategy that consists in calling everyone.
5. Estimate profit for a class decision threshold of .3 
6.  Use the model to predict on a new dataset (without the target), then use these predictions to identify those who should be called--a contact list.
7.  Make a recommendation to the Director of Sales based on your analytic work.

**NOTE:** For the module 5 project you will predict answering for prospective customers (who have not yet been called). Consequently, the assignment requires an additional dataset, customer_data.csv, that you will need to download and move into your project folder.  Here is code to load that data into memory after you have it saved in your project folder (your working directory). This code is included in the template script.

We will again use the MegaTelCo data for this demonstration.

##Load Libraries

In this class we will be using 
- Pandas
- Scikitlearn
- Matplotlib


In [50]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error

## Getting data into Pandas

In this case we will load data from the statsmodels.org library

See the Canvas assignments and lectures for a description of the Megatelco data

**Note:** you will need to enter a code supplied by Google in the next step. 


In [44]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

df = pd.read_csv (r'/content/gdrive/MyDrive/Colab Notebooks/retail_dataset.csv')



Mounted at /content/gdrive


In [45]:
#look at the top rows
df.head(10) 

,customer_id,customer_name,address_line_1,city,state_province,postal_code,order_id,order_date,customer_id.1,order_id.1,product_id,quantity,product_id.1,product_name,product_price,total_amount,order_year_month,order_month
0,67,Gordy Matteau,59 Northland Circle,Wilmington,DE,19805.0,1001,2020-01-11,67,1001,106,3.0,106,Great value Juice,$1.99,$5.97,2020-01,January
1,201,Cort Paskerful,769 Buell Drive,Huntsville,AL,35810.0,1002,2019-11-22,201,1002,104,2.0,104,Desk,$85.98,$171.96,2019-11,November
2,45,Willetta Lumley,41 Ridgeway Alley,Irvine,CA,92612.0,1003,2019-09-17,45,1003,107,5.0,107,Dove deodorant,$3.99,$19.95,2019-09,September
3,106,Alison Batterton,489 Erie Hill,Richmond,VA,23293.0,1004,2019-12-09,106,1004,106,8.0,106,Great value Juice,$1.99,$15.92,2019-12,December
4,357,Martelle Paxeford,451 Manitowish Place,Richmond,VA,23293.0,1005,2019-06-08,357,1005,107,7.0,107,Dove deodorant,$3.99,$27.93,2019-06,June
5,351,Glenn Reville,3 Springview Drive,Dallas,TX,75277.0,1006,2020-02-18,351,1006,106,4.0,106,Great value Juice,$1.99,$7.96,2020-02,February
6,233,Cosette Berkowitz,7619 Susan Lane,South Bend,IN,46634.0,1007,2019-10-25,233,1007,105,2.0,105,Office Chair,$55.99,$111.98,2019-10,October
7,121,Nisse Cunniam,0 Dennis Pass,Lincoln,NE,68531.0,1008,2019-11-10,121,1008,101,1.0,101,Dell E5470 Laptop,$485.00,$485.00,2019-11,November
8,232,Yank Eayrs,8176 Lawn Drive,Washington,DC,20575.0,1009,2019-09-27,232,1009,110,1.0,110,Apple iPad Mini,$329.99,$329.99,2019-09,September
9,113,Jamesy Brothwood,0693 Evergreen Pass,Scranton,PA,18514.0,1010,2019-09-02,113,1010,108,1.0,108,Printer,$59.99,$59.99,2019-09,September


In [46]:
df['total_amount'] = df['total_amount'].str.replace(',', '')
df['total_amount'] = df['total_amount'].str.replace('$', '')
df['total_amount'] = df['total_amount'].astype(float)

<ipython-input-46-a55b2f37d25b>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['total_amount'] = df['total_amount'].str.replace('$', '')


In [47]:
df['product_price'] = df['product_price'].str.replace(',', '')
df['product_price'] = df['product_price'].str.replace('$', '')
df['product_price'] = df['product_price'].astype(float)

<ipython-input-47-ef7bb9929a71>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['product_price'] = df['product_price'].str.replace('$', '')


In [48]:
df=df.dropna()

In [49]:
#look at the datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8981 entries, 0 to 9568
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       8981 non-null   int64  
 1   customer_name     8981 non-null   object 
 2   address_line_1    8981 non-null   object 
 3   city              8981 non-null   object 
 4   state_province    8981 non-null   object 
 5   postal_code       8981 non-null   float64
 6   order_id          8981 non-null   int64  
 7   order_date        8981 non-null   object 
 8   customer_id.1     8981 non-null   int64  
 9   order_id.1        8981 non-null   int64  
 10  product_id        8981 non-null   int64  
 11  quantity          8981 non-null   float64
 12  product_id.1      8981 non-null   int64  
 13  product_name      8981 non-null   object 
 14  product_price     8981 non-null   float64
 15  total_amount      8981 non-null   float64
 16  order_year_month  8981 non-null   object 


In [32]:
le = LabelEncoder()
df["state_province_idx"] = le.fit_transform(df["state_province"])
df["month_idx"] = le.fit_transform(df["order_month"])

In [33]:
#describe the data before cleaning it
df.describe()

,customer_id,postal_code,order_id,customer_id.1,order_id.1,product_id,quantity,product_id.1,product_price,total_amount,state_province_idx,month_idx
count,8981.000000,8981.000000,8981.000000,8981.000000,8981.000000,8981.000000,8981.000000,8981.000000,8981.000000,8981.000000,8981.000000,8981.000000
mean,268.464982,52230.965371,4974.766173,268.464982,4974.766173,106.688231,3.344394,106.688231,290.239500,344.002359,41.187062,5.457855
std,189.860282,29663.530626,2034.447409,189.860282,2034.447409,3.657543,4.244108,3.657543,340.809583,375.985611,26.760648,3.467014
min,1.000000,1105.000000,1001.000000,1.000000,1001.000000,101.000000,1.000000,101.000000,1.990000,1.990000,0.000000,0.000000
25%,121.000000,27404.000000,3246.000000,121.000000,3246.000000,104.000000,1.000000,104.000000,4.990000,55.990000,16.000000,2.000000
50%,243.000000,48295.000000,5430.000000,243.000000,5430.000000,107.000000,1.000000,107.000000,59.990000,167.970000,38.000000,6.000000
75%,362.000000,79923.000000,6675.000000,362.000000,6675.000000,110.000000,4.000000,110.000000,694.970000,694.970000,65.000000,8.000000
max,800.000000,99812.000000,8147.000000,800.000000,8147.000000,114.000000,20.000000,114.000000,988.720000,1977.440000,89.000000,11.000000


In [29]:
x=df[['quantity','product_price']]

In [30]:
y=df[['total_amount']]

In [52]:
model = sm.OLS(y, x).fit()
predictions = model.predict(x) # make the predictions by the model
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           total_amount   R-squared (uncentered):                   0.926
Model:                            OLS   Adj. R-squared (uncentered):              0.926
Method:                 Least Squares   F-statistic:                          5.645e+04
Date:                Wed, 12 Apr 2023   Prob (F-statistic):                        0.00
Time:                        22:52:34   Log-Likelihood:                         -57016.
No. Observations:                8981   AIC:                                  1.140e+05
Df Residuals:                    8979   BIC:                                  1.141e+05
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
quantity         11.4732      0.273     42.037      0.000      10.938      12.008
product_price     1.0674      0.003    323.986      0.000       1.061       1.074
==============================================================================
Omnibus:                     7593.691   Durbin-Watson:                   1.969
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           169007.983
Skew:                           4.140   Prob(JB):                         0.00
Kurtosis:                      22.572   Cond. No.                         83.7
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)


In [35]:
model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [38]:
predictions = model.predict(x_test)

In [40]:
test_predictions = model.predict(x_test)

MAE = mean_absolute_error(y_test,test_predictions)
MSE = mean_squared_error(y_test,test_predictions)
RMSE = np.sqrt(MSE)

print("MAE: %f" % (MAE))
print("RMSE: %f" % (RMSE))

MAE: 71.922374
RMSE: 139.283415


In [41]:
residuals = y_test - predictions
sns.scatterplot(x=y_test,y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel("Charges")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

ValueError: ignored